<a href="https://colab.research.google.com/github/kilkuwu/pytorch-deep-learning/blob/main/lessons/04/exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. Template bài tập

In [ ]:
# Check for GPU
!nvidia-smi

In [ ]:
# Import torch
import torch
from torch import nn

# Exercises require PyTorch > 1.10.0
print(torch.__version__)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Các mô hình của chúng ta đang hoạt động kém (không phù hợp tốt với dữ liệu). Hỏi 3 phương pháp để ngăn ngừa underfitting là gì? Liệt kê và giải thích.

## 2. Tái tạo các hàm tải dữ liệu chúng ta đã xây dựng trong [phần 1, 2, 3 và 4 của notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/). Bạn nên có `DataLoader` huấn luyện và kiểm tra.

In [ ]:
# 1. Get data


In [ ]:
# 2. Become one with the data
import os
def walk_through_dir(dir_path):
  """Walks through dir_path returning file counts of its contents."""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
# Setup train and testing paths


In [ ]:
# Visualize an image


In [ ]:
# Do the image visualization with matplotlib


Chúng ta đã có một số hình ảnh trong các thư mục của mình.

Bây giờ chúng ta cần làm cho chúng tương thích với PyTorch bằng cách:
1. Biến đổi dữ liệu thành tensor.
2. Biến dữ liệu tensor thành `torch.utils.data.Dataset` và sau đó thành `torch.utils.data.DataLoader`.

In [ ]:
# 3.1 Transforming data with torchvision.transforms


In [ ]:
# Write transform for turning images into tensors


In [ ]:
# Write a function to plot transformed images


### Tải dữ liệu hình ảnh bằng `ImageFolder`

In [ ]:
# Use ImageFolder to create dataset(s)


In [ ]:
# Get class names as a list


In [ ]:
# Can also get class names as a dict


In [ ]:
# Check the lengths of each dataset


In [ ]:
# Turn train and test Datasets into DataLoaders


In [ ]:
# How many batches of images are in our data loaders?


## 3. Tái tạo `model_0` chúng ta đã xây dựng trong phần 7 của notebook 04.

## 4. Tạo các hàm huấn luyện và kiểm tra cho `model_0`.

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):

  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader and data batches

    # Send data to target device

    # 1. Forward pass

    # 2. Calculate and accumulate loss


    # 3. Optimizer zero grad


    # 4. Loss backward


    # 5. Optimizer step


    # Calculate and accumualte accuracy metric across all batches


  # Adjust metrics to get average loss and average accuracy per batch


In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):

  # Put model in eval mode
  model.eval()

  # Setup the test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager

    # Loop through DataLoader batches

      # Send data to target device


      # 1. Forward pass


      # 2. Calculuate and accumulate loss


      # Calculate and accumulate accuracy


  # Adjust metrics to get average loss and accuracy per batch



In [ ]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

  # Create results dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # Loop through the training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    # Train step
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer)
    # Test step
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn)

    # Print out what's happening
    print(f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
    )

    # Update the results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the results dictionary
  return results

## 5. Thử huấn luyện mô hình bạn tạo trong bài tập 3 cho 5, 20 và 50 epoch, điều gì xảy ra với kết quả?
* Sử dụng `torch.optim.Adam()` với tốc độ học 0.001 làm optimizer.

In [ ]:
# Train for 5 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             lr=0.001)


In [ ]:
# Train for 20 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             lr=0.001)

In [ ]:
# Train for 50 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             lr=0.001)

Có vẻ như mô hình của chúng ta đang bắt đầu overfit về cuối (hoạt động tốt hơn nhiều trên dữ liệu huấn luyện so với dữ liệu kiểm tra).

Để khắc phục điều này, chúng ta phải giới thiệu các cách ngăn ngừa overfitting.

## 6. Tăng gấp đôi số lượng hidden unit trong mô hình của bạn và huấn luyện nó trong 20 epoch, điều gì xảy ra với kết quả?

In [16]:
# Double the number of hidden units and train for 20 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Có vẻ như mô hình vẫn đang overfitting, ngay cả khi thay đổi số lượng hidden unit.

Để khắc phục điều này, chúng ta phải tìm cách ngăn ngừa overfitting với mô hình của chúng ta.

## 7. Tăng gấp đôi dữ liệu bạn đang sử dụng với mô hình của bạn từ bước 6 và huấn luyện nó trong 20 epoch, điều gì xảy ra với kết quả?
* **Lưu ý:** Bạn có thể sử dụng [notebook tạo dữ liệu tùy chỉnh](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/04_custom_data_creation.ipynb) để mở rộng tập dữ liệu Food101 của bạn.
* Bạn cũng có thể tìm [tập dữ liệu gấp đôi đã được định dạng (20% thay vì 10% tập con) trên GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi_20_percent.zip), bạn sẽ cần viết mã tải xuống như trong bài tập 2 để đưa nó vào notebook này.

In [ ]:
# Download 20% data for Pizza/Steak/Sushi from GitHub
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi_20_percent"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi_20_percent.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    print("Downloading pizza, steak, sushi 20% data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi_20_percent.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi 20% data...")
    zip_ref.extractall(image_path)

In [ ]:
# See how many images we have
walk_through_dir(image_path)

Tuyệt vời, bây giờ chúng ta có gấp đôi số lượng hình ảnh huấn luyện và kiểm tra...

In [ ]:
# Create the train and test paths
train_data_20_percent_path = image_path / "train"
test_data_20_percent_path = image_path / "test"

train_data_20_percent_path, test_data_20_percent_path

In [ ]:
# Turn the 20 percent datapaths into Datasets and DataLoaders
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

simple_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create datasets


# Create dataloaders


In [ ]:
# Train a model with increased amount of data
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## 8. Thực hiện dự đoán trên hình ảnh tùy chỉnh pizza/bít tết/sushi của riêng bạn (bạn thậm chí có thể tải xuống một cái từ internet) với mô hình đã huấn luyện từ bài tập 7 và chia sẻ dự đoán của bạn.
* Mô hình bạn huấn luyện trong bài tập 7 có đúng không?
* Nếu không, bạn nghĩ bạn có thể làm gì để cải thiện nó?